In [3]:
import os
import numpy as np
import pandas as pd
import wfdb
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import seaborn as sns
from app.ecg.ecg import Datasets, EcgSignal    

ptbxl_data = pd.read_csv(r'C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\labels.csv', index_col=0)
# ptbxl_statements = pd.read_csv(Datasets.ptbxl_scp_statements.path, index_col=0)
# ptbxl_data['patient_id'] = ptbxl_data['patient_id'].astype(int)
# ptbxl_data['nurse'] = ptbxl_data['nurse'].astype('Int64')
# ptbxl_data['site'] = ptbxl_data['site'].astype('Int64')
# ptbxl_data['validated_by'] = ptbxl_data['validated_by'].astype('Int64')


In [4]:
ptbxl_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21388 entries, 15709.0 to 11744.0
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           21388 non-null  float64
 1   sex                           21388 non-null  int64  
 2   height                        6918 non-null   float64
 3   weight                        9366 non-null   float64
 4   nurse                         19949 non-null  float64
 5   site                          21372 non-null  float64
 6   device                        21388 non-null  object 
 7   recording_date                21388 non-null  object 
 8   report                        21388 non-null  object 
 9   scp_codes                     21388 non-null  object 
 10  heart_axis                    13228 non-null  object 
 11  infarction_stadium1           5600 non-null   object 
 12  infarction_stadium2           103 non-null    object 
 13

In [5]:

def get_file_paths(filename):
    filename_lr = filename.split('/')[-1].split('_')[0]
    return fr"C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\{filename_lr}.npy"

clean_tags = lambda x: [e.replace("'", "")  for e in x[1:-1].split(', ')]
ptbxl_data["file_paths"] = ptbxl_data["filename_lr"].apply(get_file_paths)
ptbxl_data["superdiagnostic"] = ptbxl_data["superdiagnostic"].apply(clean_tags)

In [6]:
ptbxl_data

,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,...,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,scp_codes_len,superdiagnostic,superdiagnostic_len,file_paths
patient_id,,,,,,,,,,,,,,,,,,,,,
15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,3,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\00001.npy
13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",...,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\00002.npy
20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\00003.npy
17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\00004.npy
17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\00005.npy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,"ventrikulÄre extrasystole(n) sinustachykardie linkstyp mÄssige amplitudenkriterien fÜr linkshypertrophie st & t abnorm, wahrscheinlich anterolaterale ischÄmie oder linksbelastung 4.46 unbestÄtigter bericht","{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'STACH': 0.0}",...,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr,4,[STTC],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\21833.npy
20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm inferiorer infarkt wahrscheinlich alt 4.46 unbestÄtigter bericht,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr,3,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\21834.npy
19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,"sinusrhythmus lagetyp normal t abnorm in anterioren ableitungen 4.46 unbestÄtigter bericht Edit: INJAS 50, (ISCAS)","{'ISCAS': 50.0, 'SR': 0.0}",...,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr,2,[STTC],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\21835.npy


# Data preparation

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(ptbxl_data["superdiagnostic"].values)
mlb.classes_.tolist()

['CD', 'HYP', 'MI', 'NORM', 'STTC']

In [8]:
train_labels = mlb.transform(ptbxl_data["superdiagnostic"].tolist())
ptbxl_data[mlb.classes_.tolist()] = train_labels

# ECGNet

In [ ]:
ecg_idx = pd.read_csv(r'C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\ptbxl_database.csv')
ptbxl_data['ecg_id']=ecg_idx['ecg_id']

In [9]:
from app.ecg.ecg import EcgDataset

valid_df = ptbxl_data[ptbxl_data['strat_fold'] == 9]
test_df = ptbxl_data[ptbxl_data['strat_fold'] == 10]

dataset = EcgDataset(ptbxl_data)
valid_dataset = EcgDataset(valid_df)
test_dataset = EcgDataset(test_df)


CQT kernels created, time used = 0.0044 seconds
CQT kernels created, time used = 0.0041 seconds
CQT kernels created, time used = 0.0052 seconds


C:\Users\redmi\AppData\Local\pypoetry\Cache\virtualenvs\ecg-service-WR4KS9pF-py3.12\Lib\site-packages\nnAudio\utils.py:429: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn("If fmax is given, n_bins will be ignored", SyntaxWarning)
C:\Users\redmi\AppData\Local\pypoetry\Cache\virtualenvs\ecg-service-WR4KS9pF-py3.12\Lib\site-packages\nnAudio\utils.py:429: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn("If fmax is given, n_bins will be ignored", SyntaxWarning)
C:\Users\redmi\AppData\Local\pypoetry\Cache\virtualenvs\ecg-service-WR4KS9pF-py3.12\Lib\site-packages\nnAudio\utils.py:429: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn("If fmax is given, n_bins will be ignored", SyntaxWarning)


In [ ]:
import torch
from sklearn.metrics import roc_auc_score
from models.ecgnet import ECGNet
import matplotlib.pyplot as plt
import joblib
import numpy as np

model = ECGNet()
model = model.double()
model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\ecgnet_weights_authors.pt'
model.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')))
model.eval()

In [ ]:
ecg, label = valid_dataset[151]
label

In [ ]:
prediction = model(ecg)
prediction

In [ ]:
softmaxed_pred = torch.softmax(prediction, dim=1).to(torch.float64)

In [ ]:
print(softmaxed_pred.detach().numpy(), label.numpy().astype(int))

In [ ]:
roc_auc_score(label.numpy().astype(int), prediction.detach().numpy().reshape(-1), average='macro')

In [ ]:
len(test_dataset)

In [ ]:
gt_all = []
pred_all = []

for id in tqdm(range(len(test_dataset))):
    ecg , label = test_dataset[id]
    # print(ecg, label)
    prediction = model(ecg)
    pred_all.append(prediction.detach().numpy())
    gt_all.append(label.detach().numpy().astype(int))
gt_all_array = np.vstack(gt_all)
pred_all_array = np.vstack(pred_all)


In [ ]:
gt_all_array

In [32]:
from models.metrics import Metrics, metric_summary, AUC, roc_auc_score
def evaluate_model(model, loader):
    gt_all = []
    pred_all = []
    for id in tqdm(range(len(loader))):
        ecg , label = loader[id]
        # print(ecg, label)
        prediction = model(ecg)
        pred_all.append(prediction.detach().numpy())
        gt_all.append(label.detach().numpy().astype(int))
    gt_all_array = np.vstack(gt_all)
    pred_all_array = np.vstack(pred_all)
    roc_score = roc_auc_score(gt_all_array, pred_all_array, average="macro")
    acc, mean_acc = Metrics(gt_all_array, pred_all_array)
    class_auc = AUC(gt_all_array, pred_all_array)
    summary = metric_summary(gt_all_array, pred_all_array)
    print(f"class wise accuracy: {acc}")
    print(f"accuracy: {mean_acc}")
    print(f"roc_score : {roc_score}")
    print(f"class wise AUC : {class_auc}")
    print(f"F1 score (Max): {summary[0]}")
    print(f"class wise precision, recall, f1 score : {summary}")
    return gt_all_array, pred_all_array
    

## EcgNet scores

In [ ]:


Metrics(gt_all_array, pred_all_array)

In [ ]:
print("ecgNet(ptbxlv1.0.2) - test - aurroc- macro: ", roc_auc_score(gt_all_array, pred_all_array))

In [ ]:
roc_score = roc_auc_score(gt_all_array, pred_all_array, average="macro")
acc, mean_acc = Metrics(gt_all_array, pred_all_array)
class_auc = AUC(gt_all_array, pred_all_array)
summary = metric_summary(gt_all_array, pred_all_array)

print(f"class wise accuracy: {acc}")
print(f"accuracy: {mean_acc}")
print(f"roc_score : {roc_score}")
print(f"class wise AUC : {class_auc}")
print(f"F1 score (Max): {summary[0]}")
print(f"class wise precision, recall, f1 score : {summary}")

In [ ]:
pred_all_array

In [ ]:
pred_all_array.shape

In [ ]:
print("ecgNet - valid - aurroc- macro: ", roc_auc_score(np.vstack(gt_all), np.vstack(pred_all)))

# ResNet50 CQT

In [31]:
from models.nn.custom_model_cqt import CustomModel
from models.config import CqtCFG
import torch

model_resnet50cqt = CustomModel(CqtCFG)
# model = model.double()
model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\resnet50\resnet50d_fold9_best_score.pth'
model_resnet50cqt.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')), strict=False)
model_resnet50cqt.eval()


CustomModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(1, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          

In [ ]:
valid_dataset = EcgDataset(valid_df, feature='cqt')
test_dataset = EcgDataset(test_df, feature='cqt')

In [ ]:
cqt_im, label = valid_dataset[0]
# cqt_im = cqt_im[None, None, :]
cqt_im.shape

In [ ]:
model_resnet50cqt(cqt_im)

In [33]:
gt_v, labels_v = evaluate_model(model_resnet50cqt, valid_dataset)

100%|██████████| 2146/2146 [03:19<00:00, 10.74it/s]

class wise accuracy: [0.7688723205964585, 0.874650512581547, 0.7483690587138863, 0.554520037278658, 0.7539608574091333]
accuracy: 0.7400745573159366
roc_score : 0.48158220540497876
class wise AUC : [0.4902048957167067, 0.6007075246769349, 0.410993496609935, 0.431645280265165, 0.4743598297561523]
F1 score (Max): 0.39902638981000393
class wise precision, recall, f1 score : (0.39902638981000393, 0.48158220540497876, [0.39902638981000393, nan, nan, nan, nan, nan, nan, nan, nan, nan], [0.2630475302889096, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.8260329294812053, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0])



C:\Users\redmi\PycharmProjects\ecg-tool-api\models\metrics.py:145: RuntimeWarning: invalid value encountered in divide
  2


In [34]:
gt_t, labels_t = evaluate_model(model_resnet50cqt, test_dataset)

100%|██████████| 2158/2158 [03:06<00:00, 11.55it/s]


class wise accuracy: [0.7701575532900834, 0.8785912882298424, 0.7451343836886005, 0.5537534754402225, 0.7585727525486562]
accuracy: 0.7412418906394811
roc_score : 0.4783042559310203
class wise AUC : [0.4759435289779123, 0.5896564080265404, 0.4259294436906377, 0.42449719104784994, 0.4754947079121608]
F1 score (Max): 0.3997443915639605
class wise precision, recall, f1 score : (0.3997443915639605, 0.4783042559310203, [0.3997443915639605, nan, nan, nan, nan, nan, nan, nan, nan, nan], [0.26309082483781276, nan, nan, nan, nan, nan, nan, nan, nan, nan], [0.8317886932344764, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0])


In [21]:
len(valid_dataset)

2146

# Xresnet1d

In [1]:
from models.nn.xresnet1d import xresnet1d101
import torch

xresnet1d_model = xresnet1d101(input_channels=12, num_classes=5)

C:\Users\redmi\AppData\Local\pypoetry\Cache\virtualenvs\ecg-service-WR4KS9pF-py3.12\Lib\site-packages\fastcore\script.py:32: SyntaxWarning: invalid escape sequence '\.'
  x = re.sub("(enum |class|function|__main__\.|\ at.*)", '', x)
C:\Users\redmi\AppData\Local\pypoetry\Cache\virtualenvs\ecg-service-WR4KS9pF-py3.12\Lib\site-packages\fastcore\script.py:33: SyntaxWarning: invalid escape sequence '\ '
  x = re.sub("(<|>|'|\ )", '', x) # spl characters
C:\Users\redmi\AppData\Local\pypoetry\Cache\virtualenvs\ecg-service-WR4KS9pF-py3.12\Lib\site-packages\fastcore\docments.py:62: SyntaxWarning: invalid escape sequence '\s'
  _clean_re = re.compile('^\s*#(.*)\s*$')


In [12]:
xresnet1d_model.double()
xresnet1d_model.eval()

XResNet1d(
  (0): ConvLayer(
    (0): Conv1d(12, 32, kernel_size=(5,), stride=(2,), padding=(2,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convs): Sequential(
        (0): ConvLayer(
          (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
    

In [10]:
ecg, label = valid_dataset[2]
ecg.shape

torch.Size([1, 12, 1000])

In [2]:
xresnet1d_model(ecg)

NameError: name 'xresnet1d_model' is not defined

# ResNET1d_wang

In [14]:
from models.nn.resnet1d import resnet1d_wang

resnet1d_wang_model = resnet1d_wang(input_channels=12, num_classes=5)
resnet1d_wang_model.double()
resnet1d_wang_model.eval()



ResNet1d(
  (0): Conv1d(12, 128, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
  (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Sequential(
    (0): BasicBlock1d(
      (conv1): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (4): Sequential(
    (0): BasicBlock1d(
      (conv1): Conv1d(128, 128, kernel_size=(5,), stride=(2,), padding=(2,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False

In [15]:
resnet1d_wang_model(ecg)

tensor([[-0.0151,  0.0248, -0.0078,  0.0188, -0.0117]], dtype=torch.float64,
       grad_fn=<MmBackward0>)

# Inception1d

In [1]:
from models.nn.inception1d import inception1d

inception1d_model = inception1d(input_channels=12, num_classes=5)
inception1d_model.double()
inception1d_model.eval()

Inception1d(
  (layers): Sequential(
    (0): InceptionBackbone(
      (im): ModuleList(
        (0): InceptionBlock1d(
          (bottleneck): Conv1d(12, 32, kernel_size=(1,), stride=(1,), bias=False)
          (convs): ModuleList(
            (0): Conv1d(32, 32, kernel_size=(39,), stride=(1,), padding=(19,), bias=False)
            (1): Conv1d(32, 32, kernel_size=(19,), stride=(1,), padding=(9,), bias=False)
            (2): Conv1d(32, 32, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
          )
          (conv_bottle): Sequential(
            (0): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
            (1): Conv1d(12, 32, kernel_size=(1,), stride=(1,), bias=False)
          )
          (bn_relu): Sequential(
            (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU()
          )
        )
        (1-5): 5 x InceptionBlock1d(
          (bottleneck): Conv1d(128, 32, kernel_size=(1,

In [11]:
inception1d_model(ecg)

tensor([[ 0.0840,  0.0743,  0.0098, -0.1390,  0.2182]], dtype=torch.float64,
       grad_fn=<MmBackward0>)

# RNN1d

In [17]:
from models.nn.rnn1d import RNN1d

rnn1d_model = RNN1d(input_channels=12, num_classes=5)
rnn1d_model.double()
rnn1d_model.eval()



NameError: name 'bn_drop_lin' is not defined